In [1]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
import time as tm
import pickle
import warnings
import os

In [2]:
with open( "dict", 'r' ) as f:
	words = f.read().split( '\n' )[:-1]		# Omit the last line since it is empty
	num_words = len( words )

In [3]:
n_trials = 5
t_train = 0
m_size = 0
t_test = 0
prec = 0

In [4]:
def get_bigrams( word, lim = None ):
  # Get all bigrams
  bg = map( ''.join, list( zip( word, word[1:] ) ) )
  # Remove duplicates and sort them
  bg = sorted( set( bg ) )
  # Make them into an immutable tuple and retain only the first few
  return tuple( bg )[:lim]

In [5]:
class BigramDecisionTree:
    def __init__(self):
        self.model = None
        self.bigram_index = None
        self.dictionary = None

    def _extract_bigrams(self, word):
        """Extract bigrams from a word."""
        return sorted(set([word[i:i+2] for i in range(len(word) - 1)]))[:5]

    def _create_feature_matrix(self, words):
        """Create feature matrix for words based on bigrams."""
        bigrams = set()
        word_bigrams = []

        for word in words:
            word_bigrams.append(self._extract_bigrams(word))
            bigrams.update(word_bigrams[-1])

        self.bigram_index = {bigram: idx for idx, bigram in enumerate(sorted(bigrams))}
        features = np.zeros((len(words), len(self.bigram_index)))

        for i, bigram_list in enumerate(word_bigrams):
            for bigram in bigram_list:
                features[i, self.bigram_index[bigram]] = 1

        return features

    def fit(self, dictionary):
        """Fit the decision tree model."""
        features = self._create_feature_matrix(dictionary)
        labels = np.arange(len(dictionary))
        self.model = DecisionTreeClassifier()
        self.model.fit(features, labels)
        self.dictionary = dictionary

    def predict(self, bigram_tuple):
        """Predict the word based on the bigrams."""
        feature_vector = np.zeros((1, len(self.bigram_index)))
        for bigram in bigram_tuple:
            if bigram in self.bigram_index:
                feature_vector[0, self.bigram_index[bigram]] = 1
        predicted_index = self.model.predict(feature_vector)[0]
        return [self.dictionary[predicted_index]]
################################
# Non Editable Region Starting #
################################
def my_fit( words ):
################################
#  Non Editable Region Ending  #
################################
	model = BigramDecisionTree()
	model.fit(words)
	# Do not perform any file IO in your code
	# Use this method to train your model using the word list provided

	return model					# Return the trained model


################################
# Non Editable Region Starting #
################################
def my_predict( model, bigram_list ):
################################
#  Non Editable Region Ending  #
################################
	guess_list = model.predict(bigram_list)
	# Do not perform any file IO in your code
	# Use this method to predict on a test bigram_list
	# Ensure that you return a list even if making a single guess
	return guess_list

In [7]:
lim_bg = 5
lim_out = 5

t_train = 0
t_test = 0
m_size = 0

for t in range(n_trials):
    # Record the start time for training
    tic = tm.perf_counter()

    # Train the model
    model = my_fit(words)

    # Record the end time for training
    toc = tm.perf_counter()

    # Accumulate the training time
    t_train += toc - tic

    # Save the model to a file
    with open(f"model_dump_{t}.pkl", "wb") as outfile:
        pickle.dump(model, outfile, protocol=pickle.HIGHEST_PROTOCOL)

    # Accumulate the size of the model
    m_size += os.path.getsize(f"model_dump_{t}.pkl")

    # Record the start time for testing
    tic = tm.perf_counter()

    for (i, word) in enumerate(words):
        # Get bigrams for the word
        bg = get_bigrams(word, lim=lim_bg)

        # Make predictions using the model
        guess_list = my_predict(model, bg)

        # Limit the number of guesses to be considered
        guess_len = len(guess_list)
        guess_list = guess_list[:lim_out]

        # Calculate precision if the word is in the guess list
        if word in guess_list:
            prec += 1 / guess_len

    # Record the end time for testing
    toc = tm.perf_counter()

    # Accumulate the testing time
    t_test += toc - tic


In [8]:
t_train /= n_trials
m_size /= n_trials
t_test /= n_trials
prec /= ( n_trials * num_words )

print( t_train, m_size, prec, t_test )

4.798775332600007 417122205.0 0.9750338687826592 0.7729708611999968
